# Hu and Ma (2024) Video Processing Package

This example shows how to use `interactionvideo` package to process a video to study interpersonal interactions and communications. 

Please refer to and cite the research paper: Hu and Ma (2024), "Persuading Investors: A Video-Based Study", available at [SSRN](https://papers.ssrn.com/sol3/papers.cfm?abstract_id=3583898), by [Allen Hu](https://www.allenanhu.com) (UBC Sauder, allen.hu@sauder.ubc.ca) and 
[Song Ma](https://songma.github.io) (Yale University and NBER, song.ma@yale.edu)

**For academic research purposes only.**

## Overview

The video processing involves the following steps:
1. Set up folders and check dependencies and requirements
2. Extract images and audios from a video using [`pliers`](https://github.com/PsychoinformaticsLab/pliers)
3. Extract text from audios using [Google Speech2Text API](https://cloud.google.com/speech-to-text)
4. Process images (faces) using [Face++ API](https://www.faceplusplus.com/)
5. Process text using [Loughran and McDonald (2011)](https://onlinelibrary.wiley.com/doi/abs/10.1111/j.1540-6261.2010.01625.x) Finance Dictionary and [Nicolas, Bai, and Fiske (2019)](https://osf.io/preprints/psyarxiv/afm8k) Social Psychology Dictionary
6. Process audios using pre-trained ML models in [`pyAudioAnalysis`](https://github.com/tyiannak/pyAudioAnalysis) and [`speechemotionrecognition`](https://github.com/hkveeranki/speech-emotion-recognition)
7. Aggregate information from 3V (visual, vocal, and verbal) to video level

## File Structure

- `example.py` and `example.ipynb`: step-by-step tutorials
  - We recommend you start from `example.ipynb`
- `interactionvideo`: main package folder
- `data`: data input folder
- `output`: output result folder
- `mlmodel`: pre-trained ML model folder
- `PythonSDK`: Face++ Python SDK folder, downloaded directly from [Github](https://github.com/FacePlusPlus/facepp-python-sdk)

```bash
├── interactionvideo
│   ├── __pycache__
│   ├── prepare.py
│   ├── decompose.py
│   ├── faceppml.py
│   ├── googleml.py
│   ├── textualanalysis.py
│   ├── audioml.py
│   ├── aggregate.py
│   └── utils.py
├── data
│   ├── example_video.mp4
│   └── VideoDictionary.csv
├── mlmodel
│   ├── pyAudioAnalysis
│   └── speechemotionrecognition
├── output
│   ├── audio_temp
│   ├── image_temp
│   └── result_temp
├── PythonSDK
├── README.md
├── requirement.txt
├── environment.yml
├── example.py
└── example.ipynb
```

## 1. Set up folders and check dependencies

In [3]:
import os
from os.path import join

# Set the path
RootPath = ''

# Set your video file path here
VideoFilePath = join(RootPath,'data','example_video.mp4')

# Set your work path here
# Work path is where to store meta files and output files
WorkPath = join(RootPath,'output')

# Set your path to the Google Cloud credential file here
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = ''

# Append path to ffmpeg
FFMPEGPath = ''
os.environ['PATH'] += os.pathsep + FFMPEGPath

In [4]:
# Set up the folders
from interactionvideo.prepare import setup_folder
setup_folder(WorkPath)

# check the requirements for interactionvideo
from interactionvideo.prepare import check_requirements
check_requirements()

decompose.py requirements satisfied.

faceppml.py requirements satisfied.

googleml.py requirements satisfied.

audioml.py requirements satisfied.



True

## 2. Extract images and audios from video

In [5]:
from interactionvideo.decompose import convert_video_to_images

# Decompose the video into a stream of images
# The default sampling rate is 10 frames per second
# Find the output at WorkPath\image_temp
convert_video_to_images(VideoFilePath, WorkPath)

Video is 70.12 seconds long.



100%|██████████| 702/702 [05:55<00:00,  1.98it/s]

Video is sampled to 702 images.

Video to images finished.



True

In [6]:
from interactionvideo.decompose import convert_video_to_audios

# Decompose the video into audios
# Find the output at WorkPath\audio_temp
convert_video_to_audios(VideoFilePath, WorkPath)

[MoviePy] Writing audio in E:\Research Dropbox\Research\VideoProcessing\Code\Git Repo\202408\Python\output\audio_temp\audio_full.wav


100%|██████████| 1683/1683 [00:00<00:00, 2770.97it/s]

[MoviePy] Done.
Video to audios finished.



True

## 4. Process images (faces) using Face++ API

Use Face++ ML API to process images.
  
Get the key and secret from https://www.faceplusplus.com. If you register at https://console.faceplusplus.com/register, use https://api-us.faceplusplus.com as the server. If you register at https://console.faceplusplus.com.cn/register, use https://api-cn.faceplusplus.com as the server. The Python SDK of Face++ is included in this repo (Python SDK). Check the orginial package at https://github.com/FacePlusPlus/facepp-python-sdk.

In [7]:
from interactionvideo.faceppml import process_image_by_facepp

# Use Face++ ML API to process images
# Return csv files of facial emotion, gender, predicted age
# Find the output
# - WorkPath\result_temp\face_panel_facepp.csv (full returns from Face++)
# - WorkPath\result_temp\face_panel.csv (clean results)

# Set your key, secret, and server here
FaceppKey = ''
FaceppSecret = ''
FaceppServer = 'https://api-us.faceplusplus.com'

facepp_result_df, facepp_result_clean_df = process_image_by_facepp(VideoFilePath, WorkPath,\
                                            FaceppKey, FaceppSecret, FaceppServer)

Face++ API begins. 702 images to process.



100%|██████████| 702/702 [1:09:10<00:00,  5.91s/it]


Face++ API ends. 702 images processed.



In [8]:
# Check full returns from Face++
facepp_result_df.head(10)

,ImageName,Onset,Offset,Duration,face_rectangle#top,face_rectangle#left,face_rectangle#width,face_rectangle#height,landmark#contour_chin#x,landmark#contour_chin#y,...,attributes#beauty#male_score,attributes#beauty#female_score,attributes#mouthstatus#surgical_mask_or_respirator,attributes#mouthstatus#other_occlusion,attributes#mouthstatus#close,attributes#mouthstatus#open,Number of Faces,Visual-Positive,Visual-Negative,Visual-Beauty
0,frame[0],0.0,0.1,0.1,405,868,249,249,1008,654,...,41.792,44.380,0.0,0.003,0.050,99.947,1,0.00007,0.26985,0.430860
1,frame[3],0.1,0.2,0.1,406,867,250,250,1007,655,...,38.823,42.515,0.0,0.000,0.000,100.000,1,0.00008,0.22857,0.406690
2,frame[6],0.2,0.3,0.1,404,866,252,252,1006,655,...,39.413,43.381,0.0,0.000,0.000,100.000,1,0.00115,0.32879,0.413970
3,frame[9],0.3,0.4,0.1,403,867,253,253,1006,655,...,38.357,42.225,0.0,0.000,0.000,100.000,1,0.00152,0.33477,0.402910
4,frame[12],0.4,0.5,0.1,401,866,258,258,1005,658,...,38.983,44.059,0.0,0.000,0.000,100.000,1,0.00040,0.92615,0.415210
5,frame[15],0.5,0.6,0.1,405,867,261,261,1010,665,...,42.981,46.557,0.0,0.000,0.000,100.000,1,0.00734,0.98612,0.447690
6,frame[18],0.6,0.7,0.1,407,867,261,261,1010,667,...,43.899,46.042,0.0,0.000,0.000,100.000,1,0.00187,0.79954,0.449705
7,frame[21],0.7,0.8,0.1,404,869,258,258,1012,662,...,42.091,47.842,0.0,0.000,0.000,99.999,1,0.00021,0.09503,0.449665
8,frame[24],0.8,0.9,0.1,403,867,262,262,1014,664,...,42.089,48.186,0.0,0.003,0.000,99.996,1,0.00095,0.61223,0.451375
9,frame[27],0.9,1.0,0.1,402,868,262,262,1014,664,...,44.724,49.031,0.0,0.001,0.069,99.930,1,0.00046,0.05613,0.468775


In [9]:
# Check clean results
facepp_result_clean_df.head(10)

,Onset,Offset,Duration,Number of Faces,Gender,Age,Visual-Positive,Visual-Negative,Visual-Beauty
0,0.0,0.1,0.1,1,Male,31,0.00007,0.26985,0.430860
1,0.1,0.2,0.1,1,Male,33,0.00008,0.22857,0.406690
2,0.2,0.3,0.1,1,Male,30,0.00115,0.32879,0.413970
3,0.3,0.4,0.1,1,Male,28,0.00152,0.33477,0.402910
4,0.4,0.5,0.1,1,Male,28,0.00040,0.92615,0.415210
5,0.5,0.6,0.1,1,Male,26,0.00734,0.98612,0.447690
6,0.6,0.7,0.1,1,Male,30,0.00187,0.79954,0.449705
7,0.7,0.8,0.1,1,Male,32,0.00021,0.09503,0.449665
8,0.8,0.9,0.1,1,Male,29,0.00095,0.61223,0.451375
9,0.9,1.0,0.1,1,Male,29,0.00046,0.05613,0.468775


## 3. Extract text from audios using Google Speech2Text API

Set up your Google Cloud environment following

 - https://cloud.google.com/python
 - https://cloud.google.com/storage/docs/quickstart-console
 - https://developers.google.com/workspace/guides/create-credentials
 - https://cloud.google.com/storage/docs/creating-buckets

Create a Google Cloud Storage bucket.

In [12]:
from interactionvideo.googleml import upload_audio_to_googlecloud

# Set your Google Cloud Storage bucket name here
GoogleBucketName = ''

# Upload audio file to Google Cloud Storage
upload_audio_to_googlecloud(WorkPath, GoogleBucketName)

Uploaded the audio file to Google Cloud.



True

In [13]:
from interactionvideo.googleml import convert_audio_to_text_by_google

# Use Google Speech2Text API to convert audio to text
# Return a txt file of full speech script and a csv file of text and punctuation
# Find the output at 
# - WorkPath\result_temp\script_google.txt (full speech script)
# - WorkPath\result_temp\text_panel_google.csv (text panel from Google)
google_result_text, google_result_df = convert_audio_to_text_by_google(WorkPath, GoogleBucketName)

Google Speech2Text begins. 70.12 seconds audio to process.

Google Speech2Text ends. 70.12 seconds audio processed.



In [14]:
# Check full speech script from Google
print(google_result_text)

Hello, everyone. First of all, we will like to thank you for your interest, in our research. In this paper, we try to understand how human interaction features such as facial expressions, vocal emotions, and word choices, might influence economic agents decision-making. In order to study this question. Empirically, we build an empirical approach that uses videos of human interactions as data input, and, and machine learning based algorithms as the tool. We applied a couple approached in a setting where early stage Startup pitch, Venture capitalists, for early stage funding. We find that pitch features along visual vocal and verbal Dimensions, all matter for the probability of receiving funding. And we also show that this event impact is largely due to interaction, induced biases, rather than that interactions provide additional valuable information. The empirical structure that you see, in this code example, can hopefully help you to get started with using video in other important sett

In [15]:
# Check text panel from Google
google_result_df.head(10)

,Text,Onset,Offset,Duration,Sentence End
0,"Hello,",0.4,0.6,0.2,True
1,everyone.,0.6,1.2,0.6,True
2,First,1.2,1.5,0.3,False
3,of,1.5,1.5,0.0,False
4,"all,",1.5,1.9,0.4,True
5,we,1.9,2.0,0.1,False
6,will,2.0,2.2,0.2,False
7,like,2.2,2.3,0.1,False
8,to,2.3,2.4,0.1,False
9,thank,2.4,2.7,0.3,False


## 5. Process text using LM and NBF Dictionaries

Use Loughran-McDonald (2011) Finance Dictionary (LM) to construct verbal positive and negative.

For more details, please check https://sraf.nd.edu/textual-analysis/resources.

Use Nicolas, Bai, and Fiske (2019) Social Psychology Dictionary (NBF) to construct verbal warmth and ability.

For more details, please check https://psyarxiv.com/afm8k.

In [16]:
from interactionvideo.textualanalysis import process_text_by_dict

# Set LM-NBF dictionary path
DictionaryPath = join(RootPath,'data','VideoDictionary.csv')

# Dictionary-based textual analysis to get verbal measures
# (e.g., verbal positive, negative, warmth, ability)
# Return csv files of verbal positive, negative, warmth, and ability
# Find the output at 
# - WorkPath\result_temp\text_panel.csv
text_result_df = process_text_by_dict(WorkPath, DictionaryPath)

LM and NBF Dictionaries loaded.

Dictionary-based textual analysis begins. 181 words to process.

Dictionary-based textual analysis ends. 181 words processed.



In [17]:
# Check text panel from Dictionary
text_result_df.head(10)

,Text,Onset,Offset,Duration,Sentence End,Verbal-Negative,Verbal-Positive,Verbal-Warmth,Verbal-Ability
0,"Hello,",0.4,0.6,0.2,True,0.0,0.0,0.0,0.0
1,everyone.,0.6,1.2,0.6,True,0.0,0.0,0.0,0.0
2,First,1.2,1.5,0.3,False,0.0,0.0,0.0,0.0
3,of,1.5,1.5,0.0,False,0.0,0.0,0.0,0.0
4,"all,",1.5,1.9,0.4,True,0.0,0.0,0.0,0.0
5,we,1.9,2.0,0.1,False,0.0,0.0,0.0,0.0
6,will,2.0,2.2,0.2,False,0.0,0.0,0.0,0.0
7,like,2.2,2.3,0.1,False,0.0,0.0,1.0,0.0
8,to,2.3,2.4,0.1,False,0.0,0.0,0.0,0.0
9,thank,2.4,2.7,0.3,False,0.0,0.0,1.0,0.0


## 6. Process audios by pre-trained ML models

Construct vocal arousal and vocal valence from pre-trained SVM ML models in `pyAudioAnalysis`.

The pre-trained models are located at mlmodel\pyAudioAnalysis
- svmSpeechEmotion_arousal
- svmSpeechEmotion_arousalMEANS
- svmSpeechEmotion_valence
- svmSpeechEmotion_valenceMEANS

For more details, please check https://github.com/tyiannak/pyAudioAnalysis/wiki/4.-Classification-and-Regression.

Construct vocal positive and vocal negative from pre-trained LSTM ML models in `speechemotionrecognition`.

The pre-trained models are located at mlmodel\speechemotionrecognition
- best_model_LSTM_39.h5

For more details, please check https://github.com/harry-7/speech-emotion-recognition.

Note: speechemotionrecognition requires tensorflow and Keras.


In [18]:
from interactionvideo.audioml import process_audio_by_pyAudioAnalysis

# Set the model path
pyAudioAnalysisModelPath = join(RootPath,'mlmodel','pyAudioAnalysis')

# Construct vocal arousal and vocal valence
# Find the output at 
# - WorkPath\result_temp\audio_panel_pyAudioAnalysis.csv
audio_result_df1 = process_audio_by_pyAudioAnalysis(WorkPath, pyAudioAnalysisModelPath)

Using TensorFlow backend.


pyAudioAnalysis vocal emotion analysis begins. 70.12 seconds audio to process.

pyAudioAnalysis ML model loaded.

pyAudioAnalysis vocal emotion analysis ends. 70.12 seconds audio processed.



In [19]:
# Check audio panel from pyAudioAnalysis
audio_result_df1.head()

,Onset,Offset,Duration,Vocal-Arousal,Vocal-Valence
0,0,70.12,70.12,0.404801,-0.015317


In [20]:
from interactionvideo.audioml import process_audio_by_speechemotionrecognition

# Set the model path
speechemotionrecognitionModelPath = join(RootPath,'mlmodel','speechemotionrecognition')

# Construct vocal positive and vocal negative
# Find the output at 
# - WorkPath\result_temp\audio_panel_speechemotionrecognition.csv
audio_result_df2 = process_audio_by_speechemotionrecognition(WorkPath, speechemotionrecognitionModelPath)

speechemotionrecognition vocal emotion analysis begins. 70.12 seconds audio to process.





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128)               86016     
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                4128      
_________________________________________________________________
dense_2 (Dense)              (None, 16)                528       
_________________________________________________________________
dense_3 (Dense

None



speechemotionrecognition ML model loaded.

speechemotionrecognition vocal emotion analysis ends. 70.12 seconds audio processed.



In [21]:
# Check audio panel from speechemotionrecognition
audio_result_df2.head()

,Onset,Offset,Duration,Vocal-Positive,Vocal-Negative
0,0,70.12,70.12,0.487744,0.0068


## 7. Aggregate information from 3V to video level

In [22]:
from interactionvideo.aggregate import aggregate_3v_to_video

# Aggregate 3V information
# Find the output at 
# - WorkPath\result_temp\video_panel.csv
video_result_df = aggregate_3v_to_video(WorkPath)

3V to video aggregation finished.



In [23]:
# Check video panel
video_result_df.T

,0
Number of Faces,1.0
Gender,Male
Age,32.0
Visual-Positive,0.014174
Visual-Negative,0.44359
Visual-Beauty,0.450598
Vocal-Positive,0.49
Vocal-Negative,0.01
Vocal-Arousal,0.4
Vocal-Valence,-0.02
